In [1]:
# Input directory:
indir <- '/home/linux/Workspace/mutating-doodle/'

# Wild type sequence provided in the "Dataset Description":
wtseq <- 'VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK'

# Read testing set sequences and pH:
test <- read.csv(paste0(indir,'test.csv'))

# Add mutation information to testing set:
test[,c('type','resid','wt','mut')] <- do.call(rbind,lapply(test$protein_sequence,function(seq){
  # case 1 = wild type:
  if(seq==wtseq){ 
    return(c('WT',-1,NaN,NaN))
  # case 2 = substitution:
  } else if(nchar(seq)==nchar(wtseq)){ 
    i <- mapply(function(x,y) which(x!=y)[1], strsplit(seq,""), strsplit(wtseq,""))
    return(c('SUB',i,substr(wtseq,i,i),substr(seq,i,i)))
  # case 3 = deletion:
  } else if(nchar(seq)<nchar(wtseq)){ 
    wtsub <- substr(wtseq,1,nchar(seq))
    i <- mapply(function(x,y) which(x!=y)[1], strsplit(seq,""), strsplit(wtsub,""))
    return(c('DEL',i,substr(wtseq,i,i),NaN))
  }
}))
head(test)

,seq_id,protein_sequence,pH,data_source,type,resid,wt,mut
,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK,8,Novozymes,SUB,17,L,E
2,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK,8,Novozymes,SUB,17,L,K
3,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK,8,Novozymes,DEL,17,L,NaN
4,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK,8,Novozymes,SUB,18,K,C
5,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK,8,Novozymes,SUB,18,K,F
6,31395,VPVNPEPDATSVENVALGTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK,8,Novozymes,SUB,18,K,G


In [2]:
# Read AlphaFold2 result for wild type sequence:
alpha <- read.table(paste0(indir,'wildtype_structure_prediction_af2.pdb'),fill=T)
alpha <- unique(alpha[alpha$V1=='ATOM',c(6,11)])
colnames(alpha) <- c('resid','b')

# Add B factor to the testing set:
test <- merge(test,alpha,all.x=T)
test <- test[order(test$seq_id),]

# Download blosum matrix and add score to testing set:
download.file('https://ftp.ncbi.nih.gov/blast/matrices/BLOSUM100', destfile="BLOSUM100.txt", method="wget")
blosum <- read.table('BLOSUM100.txt')
test$blosum <- apply(test,1,function(x){
  if(x['type']=='WT'){
    return(0)
  } else if(x['type']=='DEL'){
    return(-10)
  } else {
    return(blosum[x['wt'],x['mut']])
  }
})
test$blosum[test$blosum>0] <- 0
head(test)

,resid,seq_id,protein_sequence,pH,data_source,type,wt,mut,b,blosum
,<chr>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
907,17,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK,8,Novozymes,SUB,L,E,55.23,-7
908,17,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK,8,Novozymes,SUB,L,K,55.23,-6
909,17,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK,8,Novozymes,DEL,L,NaN,55.23,-10
1033,18,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK,8,Novozymes,SUB,K,C,69.25,-8
1034,18,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK,8,Novozymes,SUB,K,F,69.25,-6
1027,18,31395,VPVNPEPDATSVENVALGTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK,8,Novozymes,SUB,K,G,69.25,-5


In [4]:
# Run DeepDDG on http://protein.org.cn/ddg.html by uploading the PDB file and clicking "Submit":
ddg <- read.table('/home/linux/Workspace/mutating-doodle/ddgout.txt')
colnames(ddg) <- c('chain','wt','resid','mut','ddg')

# Add DeepDDG output to the testing set:
test <- merge(test,ddg,all.x=T)
test[test$type=='WT','ddg'] <- 0
test[test$type=='DEL','ddg'] <-  median(test$ddg,na.rm=T)
test <- test[order(test$seq_id),]

# Run DeMaSk on https://demask.princeton.edu/query/ by pasting the wild type sequence and clicking "Compute":
demask <- read.table('/home/linux/Workspace/mutating-doodle/demaskout.txt',header=T)
colnames(demask)[c(1:4)] <- c('resid','wt','mut','demask')

# Add DeMask output to the testing set:
test <- merge(test,demask[,c(1:4)],all.x=T)
test[test$type=='WT','demask'] <- 0
test[test$type=='DEL','demask'] <- min(test$demask,na.rm=T)
test <- test[order(test$seq_id),]
head(test)

,resid,wt,mut,seq_id,protein_sequence,pH,data_source,type,b,blosum,chain,ddg,demask
,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
907,17,L,E,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK,8,Novozymes,SUB,55.23,-7,A,-0.2260,-0.2504
908,17,L,K,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK,8,Novozymes,SUB,55.23,-6,A,-0.1690,-0.2494
909,17,L,NaN,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK,8,Novozymes,DEL,55.23,-10,NA,-1.0765,-0.5986
1027,18,K,C,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK,8,Novozymes,SUB,69.25,-8,A,-1.2770,-0.1813
1028,18,K,F,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK,8,Novozymes,SUB,69.25,-6,A,-1.3530,-0.1822
1029,18,K,G,31395,VPVNPEPDATSVENVALGTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK,8,Novozymes,SUB,69.25,-5,A,-1.7260,-0.1025


In [5]:
test$tm <- (rank(-test$b) + rank(test$blosum) + rank(test$demask) + rank(test$ddg)) / 4
write.csv(test[,c('seq_id','tm')],file='submission.csv',row.names=FALSE,quote=FALSE)
head(test)

,resid,wt,mut,seq_id,protein_sequence,pH,data_source,type,b,blosum,chain,ddg,demask,tm
,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
907,17,L,E,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK,8,Novozymes,SUB,55.23,-7,A,-0.2260,-0.2504,1445.250
908,17,L,K,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK,8,Novozymes,SUB,55.23,-6,A,-0.1690,-0.2494,1523.000
909,17,L,NaN,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK,8,Novozymes,DEL,55.23,-10,NA,-1.0765,-0.5986,924.500
1027,18,K,C,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK,8,Novozymes,SUB,69.25,-8,A,-1.2770,-0.1813,1216.250
1028,18,K,F,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK,8,Novozymes,SUB,69.25,-6,A,-1.3530,-0.1822,1316.375
1029,18,K,G,31395,VPVNPEPDATSVENVALGTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK,8,Novozymes,SUB,69.25,-5,A,-1.7260,-0.1025,1482.250
